In [188]:
import numpy as np
import pandas as pd

In [190]:
data = pd.read_csv('P3_GrantExport.csv', sep=';', index_col=0, na_values=['Nicht zuteilbar - NA'])
data = data[['University', 'Approved Amount']]
data.rename(columns={'University': 'university', 'Approved Amount': 'amount'}, inplace=True)
data.index.name = 'project'
data.amount = pd.to_numeric(data.amount, errors='coerce')
data.head()

,university,amount
project,,
1,NaN,11619.0
4,Université de Genève - GE,41022.0
5,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.0
6,Universität Basel - BS,52627.0
7,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.0


In [191]:
universities = pd.read_csv('universities.csv', index_col=0)
swiss_cantons = ['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR', 'SO', 'BS', 'BL',
                 'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD', 'VS', 'NE', 'GE', 'JU']
universities = universities[universities.canton.isin(swiss_cantons)]
universities.head()

,canton,latitude,longitude
name,,,
Université de Genève - GE,GE,46.199444,6.145116
Universität Basel - BS,BS,47.560253,7.582600
Université de Fribourg - FR,FR,46.806357,7.152006
Universität Zürich - ZH,ZH,47.374322,8.550981
Université de Lausanne - LA,VD,46.521090,6.580161


In [197]:
cantons_amounts = data.merge(universities[['canton']], left_on='university', right_index=True)#, how='left')
cantons_amounts = cantons_amounts.groupby('canton').sum()
cantons_amounts = cantons_amounts.amount
#cantons_index = pd.Index(['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR', 'SO', 'BS', 'BL',
#          'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD', 'VS', 'NE', 'GE', 'JU'], name='canton')
#cantons_amounts = cantons_amounts.reindex(cantons_index)
cantons_amounts.head()

canton
AG    1.152690e+08
BE    1.555048e+09
BS    1.392481e+09
FR    4.590737e+08
GE    1.877102e+09
Name: amount, dtype: float64

In [198]:
def linear_split(nb_classes):
    return [round(cantons_amounts.quantile(i/nb_classes)) for i in range(1, nb_classes)]

In [199]:
with open('data.js', 'w') as f:
    f.write('var universities = {}\n'.format(universities.reset_index().to_json(orient='records')))
    f.write('var cantons = {}\n'.format(cantons_amounts.to_json()))
    f.write('var scale = {}'.format(linear_split(6)))

In [1]:
%%HTML
<div style="position:relative;width:100%;height:0;padding-bottom:60%;">
<iframe src="leaflet.html" style="position:absolute;width:100%;height:100%;left:0;top:0;"></iframe></div>